### Local область видимости

In [2]:
def add_two(a):
  x = 2
  return a + x

add_two(3)

print(x)

NameError: name 'x' is not defined

### Enclosing область видимости

In [3]:
def add_four(a):
  x = 2
  def add_some():
    print("x = " + str(x))
    return a + x
  return add_some()

In [4]:
add_four(3)

x = 2


5

### Global область видимости

Переменные области видимости global – это глобальные переменные уровня модуля (модуль – это файл с расширением .py).

In [5]:
x = 4

def fun():
  print(x+3)

fun()

7


В приведенном выше коде переменная x – это global переменная. Доступ к ней можно получить из любой функции объявленной в данном модуле. Но если мы этот модуль импортируем в каком-то другом модуле, то x для него уже не будет переменной уровня global.

### Built-in область видимости

Уровень Python интерпретатора. В рамках этой области видимости находятся функции open, len и т.п., также туда входят исключения. Эти сущности доступны в любом модуле Python и не требуют предварительного импорта. Built-in – это максимально широкая область видимости.

### **Замыкание (closure) в программировании** — это функция, в теле которой присутствуют ссылки на переменные, объявленные вне тела этой функции в окружающем коде и не являющиеся ее параметрами.

Как уже было сказано выше, каждый раз, когда мы вызываем функцию, у нее создаются локальные переменные (если они у нее есть), а после завершения – уничтожаются, при очередном вызове эта процедура повторяется. Можно ли сделать так, чтобы после завершения работы функции, часть локальных переменных не уничтожалась, а сохраняла свои значение до следующего запуска? Да, это можно сделать!

Локальная переменная не будет уничтожена, если на нее где-то останется “живая” ссылка, после завершения работы функции. Эту ссылку может сохранять вложенная функция. Функции построенные по такому принципу могут использоваться для построения специализированных функций, т.е. являются фабриками функций. Далее будет рассмотрен вопрос создания и использования замыканий в Python, которые как раз и использую эту идею.

In [6]:
def mul(a):
  def helper(b):
    return a * b
  return helper

mul(5)(2)

10

In [7]:
new_mul5 = mul(5)
print(new_mul5)
print(new_mul5(2))

<function mul.<locals>.helper at 0x7fdf083c6488>
10


In [11]:
def fun1(a):
  x = a * 3
  def fun2(b):    
    return b + x
  return fun2

test_fun = fun1(4)
print(test_fun(7))

19


In [12]:
def func2():
    def func1():
        nonlocal a
        a = 42
 
    a = 29
    func1()
 
    print(a)
 
func2() #На экране увидим не 29, а 42.

42


In [13]:
def f(x):
  return x

In [14]:
def g(fun, x):
  return fun(x)

In [15]:
g(f, 6)

6

In [16]:
type(g)

function

In [17]:
h = f
h(5)

5

In [18]:
def wrapper_function():
  def hello_world():
    print('Hello world!')
  hello_world()

In [19]:
wrapper_function()

Hello world!


In [20]:
def higher_order(func):
  print('Получена функция {} в качестве аргумента'.format(func))
  func()
  return func

In [21]:
higher_order(wrapper_function)

Получена функция <function wrapper_function at 0x7fdf083c6950> в качестве аргумента
Hello world!


<function __main__.wrapper_function()>

### Декоратор — это функция, которая позволяет обернуть другую функцию для расширения её функциональности без непосредственного изменения её кода.

In [22]:
def decorator_function(func):
  def wrapper():
    print('Функция-обёртка!')
    print('Оборачиваемая функция: {}'.format(func))
    print('Выполняем обёрнутую функцию...')
    func()
    print('Выходим из обёртки')
  return wrapper

In [23]:
@decorator_function
def hello_world():
  print('Hello world!')

In [24]:
hello_world()

Функция-обёртка!
Оборачиваемая функция: <function hello_world at 0x7fdf083c66a8>
Выполняем обёрнутую функцию...
Hello world!
Выходим из обёртки


In [25]:
# синтаксический сахар:
hello_world = decorator_function(hello_world)

In [26]:
hello_world()

Функция-обёртка!
Оборачиваемая функция: <function decorator_function.<locals>.wrapper at 0x7fdf083c6ea0>
Выполняем обёрнутую функцию...
Функция-обёртка!
Оборачиваемая функция: <function hello_world at 0x7fdf083c66a8>
Выполняем обёрнутую функцию...
Hello world!
Выходим из обёртки
Выходим из обёртки


In [27]:
def benchmark(func):
    import time
    
    def wrapper():
        start = time.time()
        func()
        end = time.time()
        print('[*] Время выполнения: {} секунд.'.format(end-start))
    return wrapper

@benchmark
def fetch_webpage():
    import requests
    webpage = requests.get('https://google.com')

fetch_webpage()

[*] Время выполнения: 2.178114414215088 секунд.


In [28]:
def benchmark(func):
    import time
    
    def wrapper(*args, **kwargs):
        start = time.time()
        return_value = func(*args, **kwargs)
        end = time.time()
        print('[*] Время выполнения: {} секунд.'.format(end-start))
        return return_value
    return wrapper

@benchmark
def fetch_webpage(url):
    import requests
    webpage = requests.get(url)
    return webpage.text[:100]

webpage = fetch_webpage('https://google.com')
print(webpage)

[*] Время выполнения: 0.18486380577087402 секунд.
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ru"><head><meta content


In [29]:
def benchmark(iters):
    def actual_decorator(func):
        import time
        
        def wrapper(*args, **kwargs):
            total = 0
            for i in range(iters):
                start = time.time()
                return_value = func(*args, **kwargs)
                end = time.time()
                total = total + (end-start)
            print('[*] Среднее время выполнения: {} секунд.'.format(total/iters))
            return return_value

        return wrapper
    return actual_decorator


@benchmark(iters=10)
def fetch_webpage(url):
    import requests
    webpage = requests.get(url)
    return webpage.text[:100]

webpage = fetch_webpage('https://google.com')
print(webpage)

[*] Среднее время выполнения: 0.20396161079406738 секунд.
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ru"><head><meta content


По теме: 

https://python-scripts.com/decorators
    
https://tproger.ru/translations/demystifying-decorators-in-python/amp/
    
https://www.youtube.com/watch?v=My2UpCaN7rE
    
https://devpractice.ru/closures-in-python/    